# 1. 추천 시스템
- 콘텐츠 기반 필터링(Content based filtering)
- 협업 기반 필터링(Collaborative Filtering): 최급접 이웃필터링, 잠재요인 필터링

# 2. 콘텐츠 기반 필터링(Content based filtering)
- 사용자가 특정한 아이템을 매우 선호하는 겨웅, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
- 사용자 선호 프로파일
- 사용자가 선호하는 영화의 장르, 배우, 감독에 맞게 유사한 영화를 추천

# 3. 최근접 이웃 협업 필터링(메모리 협업)
- 취향이 비슷한 사람들을 기반으로 추천(취향이 비슷한 친구에게 물어보기)
- 사용자가 매긴 평점 정보, 상품 구매이력과 같이 사용자 행동양식을 기반으로 추천
- 목표: 사용자-아이템 평점 메트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측평가 하는 것
- 아직 안본 영화를 예측해서 취향에 맞게 추천해주기
- 최근접이웃, 잠재요인 두 방식 모두 사용자-아이템 평점 행렬 데이터에만 의지해 추천
- 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 높다.

## 메모리 협업필터링

### 사용자 기반(User-User)
- 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
- TOP-N: 사용자가 좋아하는 아이템을 추천
- 특정 사용자와 타 사용자간의 유사도(similarity)를 측정한 뒤 가장 유사도가 높은 TOP-N 사용자를 추출해 그들이 선호하는 아이템을 추천

### 아이템 기반(Item-Item)
- 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다.
- 사용자들이 아이템을 좋아하는지/싫어하는지
- 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 높다.

# 4. 잠재요인 협업 필터링
- 사용자-아이템 평점행렬 데이터 속에 숨어있는 '잠재요인'을 추출해 추천 예측

- 대규모의 다차원행렬 SVD 같은 차원감소 기법으로 분해하는 과정에서 잠재요인을 추출
- P.592-594\
원본-     사용자 - 아이템 평점 행렬: R\
매트릭스- 사용자 - 잠재 요인 행렬: P\
매트릭스- 잠재요인 - 아이템 행렬: Q.T\
내적 결과값 예측 => R = (P * QT)

### 다차원 희소행렬
- 1. 사용자-아이템 행렬을 저차원 밀집 행렬의 사용자-잠재요인 행렬
- 2. 아이템-잠재요인 행렬의 전치(잠재요인-아이템 행렬)로 분해
- 잠재요인 협력 필터링 알고리즘
- : 분해한 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성
- 사용자- 잠재 요인 행렬과 아이템-잠재요인 행렬의 전치행렬(잠재요인-아이템행렬)로 분해한 데이터 세트를 다시 내적의 곱으로 결합하면서 사용자가 예측하지 않은 아이템에 대한 평점을 도출

### 행렬분해(Matrix Factorization)
- 다차원 매트릭스를 저차원 매트릭스로 분해(SVD, NMF)
- 사용자-K차원 잠재요인행렬P와 / K차원 잠재 요인-아이템행렬(Q.T)으로 분해
- R은 N*M 차원으로 구성
- P는 M*k
- Q.T는 K*N
- Q는 아이템-잠재요인 행렬
- Q.T는 Q의 전치 행렬인 잠재요인-아이템 행렬
- R = P * Q.T

### 경사하강법(SGD)을 이용한 행렬분해 

In [15]:
# p.599
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q초기화, 잠재 요인 차원 K는 3으로 설정
R= np.array([[4, np.NaN, np.NaN, 2, np.NaN],
            [np.NaN, 5, np.NaN, 3, 1],
            [np.NaN, np.NaN, 3, 4, 4],
            [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

# P와 Q행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력한다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q= np.random.normal(scale=1./K, size=(num_items, K))

In [35]:
# get_rmse()함수는 실제 R행렬의 널이 아닌 헹렬 값의 위치 인덱스를 추출해 
# 이 인덱스에 있는 실제 R 행렬값과, 
# 분해된 P,Q를 이용해 다시 조합된 예측 행렬값의 RMSE값을 반환
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬과 P, Q.t의 내적으로 예측 R행렬 생성
    # np.dot은 행렬의 곱
    full_pred_matrix = np.dot(P, Q.T)
    
   # 실제 R행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [39]:
# SGD기반의 행렬분해 수행
# R>0인 행 위치, 열위치, 값을 non_zeros 리스트에 저장
non_zeros= [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] >0]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SDG기법으로 P와 Q매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측값의 차이인 오류 값 구함
        eij= r-np.dot(P[i,:], Q[j,:] -r_lambda * P[i,:])
        # Regularization을 반영한 SGD업데이트 공식 적용
        P[i,:]= P[i,:] + learning_rate * (eij * Q[j, :] - r_lambda * P[i,:])
        Q[j,:]= Q[j,:] + learning_rate * (eij * P[i, :] - r_lambda * Q[i,:])
        
        rmse= get_rmse(R, P, Q, non_zeros)
        if (step % 50)==0:
            print("### iteration step:", step, "rmse: ", rmse)

### iteration step: 0 rmse:  0.03234370977609321
### iteration step: 0 rmse:  0.03219884931383537
### iteration step: 0 rmse:  0.03230337616581673
### iteration step: 0 rmse:  0.03242090285045925
### iteration step: 0 rmse:  0.032356736001100735
### iteration step: 0 rmse:  0.03245721540968807
### iteration step: 0 rmse:  0.032583139226079465
### iteration step: 0 rmse:  0.03253308008561175
### iteration step: 0 rmse:  0.032688299416348585
### iteration step: 0 rmse:  0.03255862108226665
### iteration step: 0 rmse:  0.032411791203371634
### iteration step: 0 rmse:  0.03233996941538673
### iteration step: 50 rmse:  0.032259595784146965
### iteration step: 50 rmse:  0.03211545695412944
### iteration step: 50 rmse:  0.03221981314584846
### iteration step: 50 rmse:  0.032336655708839535
### iteration step: 50 rmse:  0.032273145439031654
### iteration step: 50 rmse:  0.03237387010452517
### iteration step: 50 rmse:  0.03249875221883673
### iteration step: 50 rmse:  0.03244806720217455
### i

In [40]:
# 분해된 P와 Q함수를 P*Q.T로 예측행렬 만들어 출력
pred_matrix= np.dot(P, Q.T)
np.round(pred_matrix,3)

array([[4.01 , 1.53 , 1.156, 2.014, 1.606],
       [5.674, 5.032, 0.882, 3.039, 1.046],
       [4.644, 0.471, 3.031, 4.036, 4.045],
       [5.005, 2.029, 1.032, 2.019, 1.497]])

# 5. 콘텐츠 기반의 필터링 실습 - TMDB5000영화데이터세트

## 장르 속성을 이용한 영화 콘텐츠 기반 필터링

In [182]:
# p.603
import pandas as pd
movies= pd.read_csv("C:/Users/user/머신러닝/tmdb_5000_movies.csv")
movies[:2]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [183]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [185]:
# 주요칼럼을 추출해 데이터프레임으로 변환
movies_df=movies.iloc[:, [3,17,1,18,19,8,4,7]]
movies_df[:2]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."


In [186]:
pd.set_option('max_colwidth',100)
movies_df[['genres', 'keywords']][:1]

# 개별 장르 명칭은 딕셔너리 key인 'name'으로 추출

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [106]:
movies['genres'][0]
# 딕셔너리를 리스트로 묶여있다?
# 문자열을 인덱싱 해져있음
# 문자열이 들어오면 딕셔너리로 바꿔준다

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [111]:
# 문자열을 객체로 변환(리스트)
movies_df['genres']= movies_df['genres'].apply(literal_eval)
movies_df['keywords']= movies_df['keywords'].apply(literal_eval)

C:\Users\user\AppData\Local\Temp\ipykernel_12652\3455261708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres']= movies_df['genres'].apply(literal_eval)
C:\Users\user\AppData\Local\Temp\ipykernel_12652\3455261708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords']= movies_df['keywords'].apply(literal_eval)


In [107]:
from ast import literal_eval
literal_eval('{"id": 28, "name": "Action"}')

{'id': 28, 'name': 'Action'}

In [108]:
type(literal_eval('{"id": 28, "name": "Action"}'))

# 딕셔너리 형태로 들어가져있다

dict

In [109]:
literal_eval(movies['genres'][0])[0]['name']
# 리스트니까 인덱싱이 가능함
# key가 name

'Action'

In [112]:
# 장르 뽑아내기
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [113]:
# i 뽑아내기
for i in movies_df['genres'][0]:
    print(i)

{'id': 28, 'name': 'Action'}
{'id': 12, 'name': 'Adventure'}
{'id': 14, 'name': 'Fantasy'}
{'id': 878, 'name': 'Science Fiction'}


In [114]:
# i 중에서 key가 name인 것만 보여줘라
for i in movies_df['genres'][0]:
        print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [115]:
[i ['name'] for i in movies_df['genres'][0]]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [188]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])


TypeError: string indices must be integers

In [117]:
movies_df[:2]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."


In [118]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect=CountVectorizer(min_df=0,ngram_range=(1, 2))


In [119]:
movies_df['genres_literal']=movies_df['genres'].apply(lambda x: " ".join(x))
movies_df['genres_literal']

C:\Users\user\AppData\Local\Temp\ipykernel_12652\3670477529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal']=movies_df['genres'].apply(lambda x: " ".join(x))


0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

In [120]:
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape

(4803, 276)

In [121]:
# 장르 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(genre_mat, genre_mat)

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [122]:
genre_sim = cosine_similarity(genre_mat, genre_mat)

In [123]:
import numpy as np
np.argsort(genre_sim)

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]], dtype=int64)

In [124]:
np.argsort(genre_sim)[:,::1]

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]], dtype=int64)

In [125]:
np.argsort(genre_sim)[:,::1][:1]

array([[2401, 3037, 3038, ...,  813, 3494,    0]], dtype=int64)

In [129]:
genre_sim_sorted=genre_sim.argsort()[:,::-1]
genre_sim_sorted[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

In [131]:
movies_df.iloc[np.argsort(genre_sim)[:, ::-1][0], :] 

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
3494,27549,Beastmaster 2: Through the Portal of Time,"[Action, Adventure, Fantasy, Science Fiction]",4.6,17,1.478505,"[based on novel, time travel, sequel, psychotr...","Mark Singer returns as Dar, the warrior who ca...",Action Adventure Fantasy Science Fiction
813,1924,Superman,"[Action, Adventure, Fantasy, Science Fiction]",6.9,1022,48.507081,"[saving the world, journalist, dc comics, crim...",Mild-mannered Clark Kent works as a reporter a...,Action Adventure Fantasy Science Fiction
870,8536,Superman II,"[Action, Adventure, Fantasy, Science Fiction]",6.5,629,30.515175,"[saving the world, dc comics, sequel, superher...",Three escaped criminals from the planet Krypto...,Action Adventure Fantasy Science Fiction
46,127585,X-Men: Days of Future Past,"[Action, Adventure, Fantasy, Science Fiction]",7.5,6032,118.078691,"[1970s, mutant, time travel, marvel comic, bas...",The ultimate X-Men ensemble fights a war for t...,Action Adventure Fantasy Science Fiction
...,...,...,...,...,...,...,...,...,...
3041,27322,Love Jones,"[Comedy, Drama, Romance]",8.1,12,1.000178,"[sex, ex-boyfriend, independent film, african ...",Darius Lovehall is a young black poet in Chica...,Comedy Drama Romance
3039,75900,My Week with Marilyn,[Drama],6.6,406,21.006078,"[based on novel, biography, historical figure,...",Sir Laurence Olivier is making a movie in Lond...,Drama
3038,17710,Hey Arnold! The Movie,"[Animation, Family]",5.6,62,5.856363,[],When a powerful developer named Mr. Scheck wan...,Animation Family
3037,19905,"The Goods: Live Hard, Sell Hard",[Comedy],5.4,58,3.352702,[duringcreditsstinger],Who is Don Ready? Salesman? Lover? Song Stylis...,Comedy


In [102]:
# ex) 연습 p.607
a= np.array([11,10,13,12])
print(np.sort(a))
print(np.argsort(a))
print(np.argsort(a)[::-1])
print(a[[2,3,0,1]])

[10 11 12 13]
[1 0 3 2]
[2 3 0 1]
[13 12 11 10]


In [143]:
# kids 라는 영화에 검색 조건을 걸어줌
# df로 만들고

#df=movies_df.copy()
title_movie=df[df['title']=="kids"]

In [176]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes] 

In [177]:
# 상위 10개 장르별 추천 영화
similar_movies= find_sim_movie(movies_df, genre_sim_sorted, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [ ]:
# p.610
v/(v+m)*0.6  m/(v+m)*0.4

In [151]:
C= movies_df['vote_average'].mean()
m= movies_df['vote_count'].quantile(0.6)
print(C)
print(m)

6.092171559442016
370.1999999999998


In [173]:
percentile=0.6
def weighted_vote_average(recod):
    v= record['vote_count']
    R= record['vote_average']
    
    return ((v/(v+m))*R)+ ((m/(m+v))*C)

#movies['weighted_vote']=
movies_df['vote_count']=movies_df.apply(weighted_vote_average, axis=1)

NameError: name 'record' is not defined

In [168]:
movies_df['vote_count']=movies_df['vote_count'].replace(0,1)
movies_df['vote_count']

C:\Users\user\AppData\Local\Temp\ipykernel_12652\880070793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['vote_count']=movies_df['vote_count'].replace(0,1)


0       11800
1        4500
2        4466
3        9106
4        2124
        ...  
4798      238
4799        5
4800        6
4801        7
4802       16
Name: vote_count, Length: 4803, dtype: int64

In [169]:
movies_df['vote_count'].value_counts

<bound method IndexOpsMixin.value_counts of 0       11800
1        4500
2        4466
3        9106
4        2124
        ...  
4798      238
4799        5
4800        6
4801        7
4802       16
Name: vote_count, Length: 4803, dtype: int64>

In [174]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']] 

NameError: name 'genre_sim_sorted_ind' is not defined

## Surpeise를 이용한 추천 시스템 구축

In [8]:
pip install surprise --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [6]:
data=Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\user/.surprise_data/ml-100k


In [7]:
algo= SVD(random_state=0)
algo.fit(trainset)

NameError: name 'trainset' is not defined